In [1]:
import pandas as pd
import numpy as np
import datetime

In [21]:
from datetime import datetime

In [30]:
datetime.strptime('04/04/21', '%d/%m/%y')

datetime.datetime(2021, 4, 4, 0, 0)

In [39]:
l1, l2 = [], []
l3, l4 = [], []
participe = int(input('Ingresar el nùmero de participes del plan: '))
for i in range(0, participe):
    p1 = str(input(f'Poner el nombre del participe del Plan numero {i+1}: '))
    p2 = str(input(f'Ingresar la fecha de nacimiento {p1}: '))
    p3 = str(input(f'Ingresar la fecha de alta en la empresa de {p1}: '))
    p4 = float(input(f'Ingresar el salario pensionable 2021 de {p1}: '))
    l1.append(p1)
    l2.append(p2)
    l3.append(p3)
    l4.append(p4)

pd.to_datetime(l2, infer_datetime_format=True) 
pd.to_datetime(l3, infer_datetime_format=True) 

array1 = []
for i in range(0, participe):
    array1.append(int(l2[i].split('/')[2]))

array2 = []
for i in range(0, participe):
    array2.append(int(l3[i].split('/')[2]))
an1 = [46, 38, 63, 30, 30, 24]## edad actuarial
an2 = [25, 37, 29, 26, 20, 23] ## edad entradad actuarial
edad = [x-y for(x, y) in zip(array2, array1)] ## edad actuarial 2
df = pd.DataFrame({'Participe':pd.Series(l1), 'Nacimiento':pd.Series(l2), 
                   'Alta':pd.Series(l3), 'Salario 2021':pd.Series(l4), 
                  'Edad actuarial':pd.Series(an1), 'Edad entrada':pd.Series(an2)})


Ingresar el nùmero de participes del plan: 3
Poner el nombre del participe del Plan numero 1: Antoni
Ingresar la fecha de nacimiento Antoni: 06/07/1973
Ingresar la fecha de alta en la empresa de Antoni: 04/03/1992
Ingresar el salario pensionable 2021 de Antoni: 20000
Poner el nombre del participe del Plan numero 2: Alvin
Ingresar la fecha de nacimiento Alvin: 09/11/1989
Ingresar la fecha de alta en la empresa de Alvin: 19/02/2010
Ingresar el salario pensionable 2021 de Alvin: 10000
Poner el nombre del participe del Plan numero 3: silvia
Ingresar la fecha de nacimiento silvia: 11/09/1995
Ingresar la fecha de alta en la empresa de silvia: 14/06/2019
Ingresar el salario pensionable 2021 de silvia: 25000


In [40]:
df

,Participe,Nacimiento,Alta,Salario 2021
0,Antoni,06/07/1973,04/03/1992,20000.0
1,Alvin,09/11/1989,19/02/2010,10000.0
2,silvia,11/09/1995,14/06/2019,25000.0


In [76]:
array1 = []
for i in range(0, 3):
    array1.append(int(l2[i].split('/')[2]))

In [77]:
array1

[1973, 1989, 1995]

In [78]:
array2 = []
for i in range(0, 3):
    array2.append(int(l3[i].split('/')[2]))

In [81]:
[x-y for(x, y) in zip(array2, array1)]

[19, 21, 24]

In [38]:
pd.to_datetime(s, infer_datetime_format=True) 

0   2000-03-11
1   2000-03-12
2   2000-03-13
dtype: datetime64[ns]

# Planes de pensiones

### Aportaciò definida

In [2]:
def aportacion_definida(nombre_participe, salarioP_inicial: float = 29944,
                        tipo_salario: float = .0125, rendimiento_fondo: float = .018,
                        aportacion_inicial: float = 368.77 , fondo_c: float = 1364.68,
                        crecimiento_sala: float = .015,
                        tabla='perm1', jubilacion: int = 65, capital: float = 1, x=48,m=0,
                        n: int = 120, h: int = 1, birth_year: int = 1973, sexo = 'Hombre', 
                        interes: float = .01, base_tecnica:float = .0175, 
                        fondo_capitalizacion: float = 3486.75):
#     jub_t = np.arange(0, jubilacion-x)
#     aume_sala = np.round(salarioP_inicial*(1+crecimiento_sala)**(jub_t), 2)  ## SAlario hasta la jubilaciò
#     Ct = np.round(np.insert(aume_sala*tipo_salario,0, aportacion_inicial)[:-1], 2)
#     FCt = (Ct+fondo_c)*(1+rendimiento_fondo)
    if tabla == 'perm1':
        f = 'https://raw.githubusercontent.com/Joevalencia/telma/main/telma/PER2020_Col_1er.orden.csv'
        data = pd.read_csv(f)
    elif tabla == 'perm2':
        f = 'https://raw.githubusercontent.com/Joevalencia/telma/main/telma/PER2020_Ind_2ndo.orden.csv'
        data = pd.read_csv(f)
    else:
        print('Solo se permiten tabla perm1 o tabla perm2')
    datatable = data
    frac_freq = h
    if n == 120:
        n = 120 - (x + m)
    elif n != 120:
        n = n
    else:
        print('Error! Type a new "n" ')
    L = np.repeat(1.0, 122)
    L[0] = 100000
    fem = datatable.iloc[:, 1] * np.exp(-datatable.iloc[:, 2] * (birth_year + np.arange(0, 121) - 2012))
    macho = datatable.iloc[:, 3] * np.exp(-datatable.iloc[:, 4] * (birth_year + np.arange(0, 121) - 2012))
    tabla = .5 * (fem + macho)
    if sexo == 'Hombre':
      for i in range(0, 121):
          L[i + 1] = L[i] * (1 - (macho[i] / 1000))
    elif sexo == 'Mujer':
      for i in range(0, 121):
          L[i + 1] = L[i] * (1 - (fem[i] / 1000))
    elif sexo == 'unisex':
      for i in range(0, 121):
          L[i + 1] = L[i] * (1 - (tabla[i] / 1000))
    else:
      print('Error. Ingresar "Hombre", "Mujer" o "unisex"')
    L = np.round(L, 4)
    fr = np.arange(frac_freq) / frac_freq
    jon = np.outer(L[1:], fr)
    jon1 = np.outer(L[:-1], (1 - fr))
    jj = (jon + jon1).flatten().round(1)
    t = np.arange(0, (121-jubilacion))  # Temporalidad
    t2 = np.arange(jubilacion, 121)  # Edad
    i2 = (1+interes)**t
    valor = capital * i2  # unidad que crece
    px = jj[np.arange(jubilacion, 121)]/jj[jubilacion] # px
    v3 = np.around((1+base_tecnica)**-t, 5) ## actualizaciò
    res1 = valor*px*v3  ## Valor renta actuarial
    res2 = valor*v3     ## Valor renta financiera
    suma1 = round(sum(res1), 2)  ## Suma renta actuarial
    suma2 = round(sum(res2), 2)  ## Suma renta financiera
    renta1 = fondo_capitalizacion/suma1
    renta2 = fondo_capitalizacion/suma2
    vaa = np.around((valor*renta1), 2)
    va = np.around((valor*renta2), 2)
    
    print(f'Plan del participe: {nombre_participe}')
    print('='*55)
    print(f'Crecimiento Salarial: {crecimiento_sala*100}%')
    print(f'Rentabilidad del Fondo: {rendimiento_fondo*100}%')
    print(f'Interés técnico càlculo pens. {base_tecnica*100}%')
    print('='*55)
    print(f'Derechos consolidados al {birth_year+jubilacion}: {fondo_capitalizacion}')
    #print('=' * 55)
    df1 = pd.DataFrame({'t':t, 'edad':t2, 'creciente':valor, 'lx':jj[np.arange(jubilacion, 121)], 
                       'probabilidad':px, 'actualizaciò':v3, 'V.A.A.':res1,
                        'V.A.':res2, 'ACTUARIAL':vaa, 'FINANCIERA':va})

      
    #print('=' * 55)
    return df1


# $$Marta$$

In [3]:
aportacion_definida('Marta', sexo='Mujer', birth_year=1957)


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

## $$Joan$$

In [137]:
aportacion_definida('Joan', 'Hombre', fondo_capitalizacion=10140.67, 
                   birth_year=1973)

Plan del participe: Joan
Derechos consolidados al 2038: 10140.67


,t,edad,creciente,lx,probabilidad,actualizaciò,V.A.A.,V.A.,ACTUARIAL,FINANCIERA
0,0,65,1.000000,92917.1,1.000000,1.00000,1.000000,1.000000,402.73,220.35
1,1,66,1.010000,92494.7,0.995454,0.98280,0.988116,0.992628,406.75,222.56
2,2,67,1.020100,92059.2,0.990767,0.96590,0.976217,0.985315,410.82,224.78
3,3,68,1.030301,91616.2,0.985999,0.94929,0.964361,0.978054,414.93,227.03
4,4,69,1.040604,91169.5,0.981192,0.93296,0.952582,0.970842,419.08,229.30
5,5,70,1.051010,90718.8,0.976341,0.91691,0.940882,0.963682,423.27,231.59
6,6,71,1.061520,90259.3,0.971396,0.90114,0.929216,0.956578,427.50,233.91
7,7,72,1.072135,89781.8,0.966257,0.88564,0.917486,0.949526,431.78,236.25
8,8,73,1.082857,89275.2,0.960805,0.87041,0.905587,0.942529,436.10,238.61
9,9,74,1.093685,88721.5,0.954846,0.85544,0.893337,0.935582,440.46,241.00


# $$Marc$$

In [138]:
aportacion_definida('Marc', 'Hombre', fondo_capitalizacion=38948.44, 
                   birth_year=1990)

Plan del participe: Marc
Derechos consolidados al 2055: 38948.44


,t,edad,creciente,lx,probabilidad,actualizaciò,V.A.A.,V.A.,ACTUARIAL,FINANCIERA
0,0,65,1.000000,95629.4,1.000000,1.00000,1.000000e+00,1.000000,1459.84,846.34
1,1,66,1.010000,95331.3,0.996883,0.98280,9.895337e-01,0.992628,1474.43,854.80
2,2,67,1.020100,95025.7,0.993687,0.96590,9.790944e-01,0.985315,1489.18,863.35
3,3,68,1.030301,94716.8,0.990457,0.94929,9.687208e-01,0.978054,1504.07,871.98
4,4,69,1.040604,94407.2,0.987219,0.93296,9.584340e-01,0.970842,1519.11,880.70
5,5,70,1.051010,94096.8,0.983974,0.91691,9.482372e-01,0.963682,1534.30,889.51
6,6,71,1.061520,93781.4,0.980675,0.90114,9.380928e-01,0.956578,1549.65,898.40
7,7,72,1.072135,93454.4,0.977256,0.88564,9.279299e-01,0.949526,1565.14,907.39
8,8,73,1.082857,93108.1,0.973635,0.87041,9.176792e-01,0.942529,1580.79,916.46
9,9,74,1.093685,92728.4,0.969664,0.85544,9.072004e-01,0.935582,1596.60,925.63


In [12]:
t = np.arange(0, (121-65)) 
ok = np.round(29944*(1.015)**t, 2)
ok2 = np.round(ok * .0125, 2)


474.98

[Zig Zag Sum Python][1]


  [1]: https://i.stack.imgur.com/LERzk.png

Hi everyone, 

I am struggling with a tricky problem in Python.

As you may see in the image I need to obtain the value evidenced in yellow. 

The starting point is the following:

1) I have two Array, say D and E. Moreover I have a constant, say 1.018.
2) I have two starting values, say 379.91 (D6) and 1738.67(E5).
3) The result of the following value 2156.72 (E6) is obtained as sum of (D6+E5)*1.018.

My question is how to get the final value 10407.44 (E22)?

The pattern to adopt is always

E7 = (D7+E6)*1.018.

E8 = (D8+E7)*1.018.

E9 = (D9+E8)*1.018.

E10 = (D10+E9)*1.018.



E22 = sum(E4:E21)

Thanks in advance for your suggestion

In [72]:
cuantia = 1738.68
ok3 = (cuantia+ok2[1])*1.018
ok4 = pd.Series([])
for i in range(0,16):
    ok4[i] = (ok3+ok2[i+2])*1.018


<ipython-input-72-0fdac0ca3892>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ok4 = pd.Series([])


In [81]:
ok4 = np.cumsum(1738.68+pd.Series(ok2[1:17]))*1.018
ok4

0      2156.72462
1      4319.25184
2      6487.67328
3      8662.07038
4     10842.53476
5     13029.15804
6     15222.03184
7     17421.24778
8     19626.89748
9     21839.08274
10    24057.90536
11    26283.45696
12    28515.83934
13    30755.16448
14    33001.52400
15    35255.02988
dtype: float64

In [76]:
a = np.array([10,5,5,5,3])
i = 1.05
b = np.cumsum(a*i)
b  # I dunno wanna this

array([10.5 , 15.75, 21.  , 26.25, 29.4 ])

In [ ]:
c = [10.5, 16.275, 22.33875, 28.7, 33.29]  # I wanna this

In [ ]:
s = pd.Series([])
for k in a:
    s.append(((a[k]*i)+a[k+1])*i)


In [51]:
((a[0]*i)+a[1])*i

16.275000000000002

In [83]:
((((a[0]*i)+a[1])*i)+a[2])*i

22.338750000000005

In [84]:
((((((a[0]*i)+a[1])*i)+a[2])*i)+a[3])*i

28.705687500000007

In [86]:
((((((((a[0]*i)+a[1])*i)+a[2])*i)+a[3])*i)+a[4])*i

33.29097187500001